In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
train =pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')
test =pd.read_csv('0000000000002747_test_twitter_x_test.csv')

In [32]:
train.count()

tweet_id                  10980
airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

In [33]:
# Dropping Useless Columns

train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)

In [35]:
train.head(2)

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [36]:
#filling nan
mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [37]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['airline_sentiment'] = le.fit_transform(train['airline_sentiment'])
train.head(2)

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [39]:
import re
import nltk
from nltk.corpus import stopwords

In [40]:

def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words ))

In [41]:

train['clean_tweet'] = train['text'].apply(lambda x: tweet_to_words(x))

test['clean_tweet'] = test['text'].apply(lambda x: tweet_to_words(x))

In [42]:
train_clean_tweet=[]
for tweet in train['clean_tweet']:
    train_clean_tweet.append(tweet)
    
test_clean_tweet=[]
for tweet in test['clean_tweet']:
    test_clean_tweet.append(tweet)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features= v.transform(test_clean_tweet)

In [44]:
train_features

<10980x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 107004 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn.naive_bayes import MultinomialNB

In [46]:
nb = MultinomialNB()

In [47]:
nb.fit(train_features,train['airline_sentiment'])
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(nb.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.8437158469945355


In [48]:
test_features

<3660x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 33563 stored elements in Compressed Sparse Row format>

In [53]:
test_pred

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'neutral'], dtype=object)

In [52]:
import numpy as np
test_pred = nb.predict(test_features)
test_pred = le.inverse_transform(test_pred)
print(test_pred)
np.savetxt("test_pred.csv",test_pred,fmt='%s')

['negative' 'negative' 'negative' ... 'neutral' 'positive' 'neutral']
